# 🚑 Kvasir-SEG U-Net Demo

This notebook demonstrates how to load and run inference using a pretrained **U-Net** model on the **Kvasir-SEG** dataset.

In [1]:
# ✅ Install dependencies
!pip install torch torchvision albumentations opencv-python matplotlib tqdm scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
# 📁 Clone GitHub Repo
!git clone https://github.com/GaurangTari4/KvasirSeg-Unet.git
%cd KvasirSeg-Unet

Cloning into 'KvasirSeg-Unet'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 101 (delta 36), reused 11 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (101/101), 37.69 KiB | 12.56 MiB/s, done.
Resolving deltas: 100% (36/36), done.
/content/KvasirSeg-Unet


In [4]:
# 📦 Download Dataset
from dataset.download import download_and_extract
download_and_extract()

HTTPError: HTTP Error 404: Not Found

In [ ]:
# 🔽 Download pretrained model
!gdown --id 1J0TW0M7q_BW0M3Wv7NSnNdrwDuL9B8kN -O models/unet_kvasirseg.pth

In [ ]:
# 📊 Import required modules
import torch
from dataset.loader import get_dataloader
from model.unet import UNet
from utils.visualize import plot_prediction

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load validation data
val_loader = get_dataloader("./dataset", batch_size=4, split='val')

# Load model
model = UNet(in_channels=3, out_channels=1)
model.load_state_dict(torch.load("models/unet_kvasirseg.pth", map_location=device))
model = model.to(device)
model.eval()

In [ ]:
# 🔍 Run Inference on a few samples
import torch

with torch.no_grad():
    inputs, masks = next(iter(val_loader))
    inputs, masks = inputs.to(device), masks.to(device)
    preds = model(inputs)
    plot_prediction(inputs[:4].cpu(), masks[:4].cpu(), preds[:4].cpu())